# Solving Systems of Nonlinear Equations

Now that we have covered root finding algorithms for univariate scalar functions (Supplement 08), optimization algorithms for univariate scalar functions (Supplement 09), and optimization algoritms for multivariate scalar functions (Supplement 10), it's time to tackle solving systems of equations. Broadly speaking, these are classified as sets of multivariate functions.  Our goal in "solving" them is to find the values of the complete set of variables that results in each of our equations being equal to zero.  It is high dimensional root finding.  A less abstract example is something we're used to: a system of nonlinear equations:

**Important:** To be clear, in this assignment we are covering best practices for solving systems of **nonlinear equations** here.  You *can* apply these methods to a system of linear equations, but there are much more efficient, closed-form analytical solutions based on linear algebra and matrix inversions. If you are dealing with linear systems, then you want to use linear methods since they are more efficient and stable than the iterative solutions we'll use here.

$$x^2 - 3y = 14.75z - w^4$$
$$25 = x^3 - z$$
$$0 = \ln(z) - z^2 + 2x + 3$$
$$z + xw = 74$$

It is clear that we cannot solve any one equation above to determine the values for w, x, y, and z that satisfy these equations. We have to solve all four of them together, as a system, to do that.  We do have analytical methods that are useful - most students default to a substitution method, for example.  In general, this is fine if you want an analytical solution but we should note two problems with the approach.  First, solving large nonlinear systems this way requires extensive algebraic manipulations, which is both extremely tedious and extremely error-prone. Second, it isn't always possible to solve a system of nonlinear equations using analytical methods. More frequently, we will rely on numerical methods to solve nonlinear systems.  Similar to the cases we've already considered (e.g., a Newton-Raphson iteration), they are based on iterative solutions that proceed until the system reaches some convergence threshold. Just like with univariate equations, our convergence threshold is that we want the equations to evaluate to zero, it's just in this case, we want each and every equation to equal zero.

Now that we have a basic familiarity, we'll jump straight into the root finding algorithms available in Julia. Solving nonlinear systems numerically is very difficult, and it requires both good algorithms and good initial guesses, so we'll skip writing our own routines like we did with univariate functions, but it is important to know that the basic approach of iteratively updating variable values until reaching your convergence threshold is the same.

When we solve a system of equations numerically in Julia, we need to write that system as a set of equations that should evaluate to zero once we find our solution. So no matter how the functions are presented to me, I will convert them into a form where the left-hand-side is zero, and the right hand side specifies the function. Following this logic, I would write the above system as:

$$0 = -x^2 - + 3y + 14.75z - w^4$$
$$0 = x^2 - z - 25$$
$$0 = \ln(z) - z^2 + 2x + 3$$
$$0 = z + xw - 74$$

That is the form we want to work with moving forward.

Just as in root finding and optimization for scalar equations, we want to construct a function that encodes this system and returns the left hand side of our equation(s). No problem, right?  We know how to create a multivariate function, so this is straightforward.  We will walk through the way we would probably do this intuitively, and then we'll explain what that won't work for a numerical solution.

    function F(w,x,y,z)
       eq1 =  -x^2 +3*y + 14.75*z - w^4
       eq2 =   x^2 - z - 25
       eq3 =   log(z) - z^2 + 2*x + 3 #note, log(x) is ln(x); log10(x) is the common logarithm.
       eq4 =   z + x*w - 74
       return [eq1, eq2, eq3, eq4]
    end
        
Now, I can run that function for any (w,x,y,z) set and see the LHS value returned by each equation.  There is about a 0\% chance they are all equal to zero.

    eq1, eq2, eq3, eq4 = F(1,2,3,4)

Technically, there is nothing wrong with our function.  It evaluates the system.  It is just not compatible with Julia syntax for numerical root finding, so this function form is not approprite for a multivariate root finding algorithm.  We have to do two things to make this work:

1. We have to recognize that our root finding algorithm is iterative. It is going to vary the values of w, x, y, and z until it converges. As in the case with optimization of multivariate functions, when we intend to vary these values iteratively, root finding algorithms only work on a function that takes a single argument. That means we will pack w, x, y, and z into a single vector variable, just like we did in Supplement 10.
2. Similar to the way our root finding algorithm wants all of our unknown variables input as a vector-like quantity, it wants our function to return the left-hand-side solution to each equation in a vector-like set of identical size to our set of variables.

Taking both of these things together, we are now working with a multivariate vector function. It is multivariate because it depends on 4 different independent variables, and it is a vector function because it returns a vector quantity instead of a scalar quantity. It's easy to do, we just need to understand the two above points, which is what the root finding algorithm needs from us.  With that in mind, we'll construct our system as a multivariate vector function (with a vector argument). 

    function equations(var) 
    #the name "var" is arbitrary; it is the  variable that contains w, x, y, and z
        
        #We'll work with our "vector" argument var
        
        w = var[1]   #Relabel elements of the vector var so I can recognize the equations
        x = var[2]
        y = var[3]
        z = var[4]
        
        #Define our "vector" output; in general, I prefer either columns or 1D arrays.
        
        LHS1 =  -x^2 + 3*y + 14.75*z - w^4
        LHS2 =   x^2 - z - 25
        LHS3 =   log(z) - z^2 + 2*x + 3
        LHS4 =   z + x*w - 74
        
        F = [LHS1, LHS2, LHS3, LHS4] #Stack LHS values in a 1D Array
        return F #return the array of function evaluations
    end

Now, if we want to run this function for the same input as last time, we need to provide a vector argument.  

    equations([1,2,3,4]) #Use a 4-element column consistent with my F return structure.

Now that we've constructed the function, we can shift to working with Julia packages for solving nonlinear systems of equations.  These are in the NLsolve package, so let's first install that.

    using Pkg
    Pkg.add("NLsolve")


https://github.com/JuliaNLSolvers/NLsolve.jl

Next declare usage:

    using NLsolve

Now that we've loaded the NLsolve package, we will use the subroutine **nlsolve**. The syntax for the root finding algorithm is straightforward.  nlsolve has a very similar syntax to fzero (Assignment 08), but is designed for handling multivariate vector functions. As with fzero and minimization routines, you can pass options and select different solver algorithms using keyword arguments. 

Note that this problem is extremely difficult to solve, but nlsolve handles it pretty easily as long as you have decent initial guesses; I find it helps to turn on Autoscaling.  I'll also set the MaxIterations to 1000, and I'll ask it to return "info."  If I get info = 1, that means "true" - the solver converged. If I get info = 0, that means "false," the solver did not converge. Try taking your max iterations down to 10 for an illustration of nonconvergence.

    var0 = [5.,5.,5.,5.] #This is a vector with my 4 initial guesses at the "roots" for w, x, y, and z; must be floats
    solution = nlsolve(equations, var0)
    fvals    = equations(solution.zero)
    println(solution)
    fieldnames(NLsolve.SolverResults)
    println(solution.zero)
    println(solution.method)
    println(solution.x_converged)
    println(fvals)
    
Note that this returns a solution structure, and we can access the various fields in that structure using the basic syntax of structurename.fieldname. The above uses the fieldnames on the object type (NLsolve.SolverResults) to print out all of the fields in the structure I've called "solution".

**A Word of Caution**

Solving algebraic systems of nonlinear equations using numerical methods is incredibly difficult; you need a good algorithm and good-to-very good initial guesses. Unfortunately, it can be difficult to provide good initial guesses. We should use our physical insights about the systems we are studying to make initial guesses that we think will be very close to the actual solution, otherwise, it is unlikely that our solvers will converge for difficult nonlinear systems. This example is actually extremely difficult to solve.  Usually, the problems we'll have in 786 are more friendly than this one, but it goes to show how hard it can be to find a numerical solution to a system of nonlinear equations.

**One final useful skill**

Providing a Jacobian is usually a good idea, and it can be essential to performance and stability for difficult systems.  Many numerical methods in Julia will accept a Jacobian as an input.  Since it is such an essential skill, we'll demonstrate its usage.  The Jacobian is the set of partial derivatives of each function in your system with respect to each variable. In the system above, that means my Jacobian will be a 4x4 matrix since I'd have to take a partial derivative of each equation with respect to w, x, y, and z.  It isn't *hard*, but it is tedious (although you may want to look at symbolic and numerical methods for generating Jacobians, which can make it much easier.  We won't cover that here). We'll go through a simpler example and create the corresponding Jacobian.

Here's a simple system of equations written as functions of (a,b)

$$0 = a^3 - e^{-b}$$
$$0 = ab - b^2 + 5$$

As a multivariate vector function:

    function q(v)
        a = v[1]
        b = v[2]
        f1 = a^3 - exp(-b)
        f2 = a*b - b^2 + 5
        F = [f1; f2]
    end
        
Now for the jacobian, we need to calculate partial derivatives. a is my first variable, b is my second variable.  So I need to create a matrix that is:

$$J = \begin{vmatrix}
\frac{df_1}{da} & \frac{df_1}{db} \\
\frac{df_2}{da} & \frac{df_2}{db}
\end{vmatrix}$$

In other words, the first row contains the partial derivatives of the first function with respect to each variable, the second row contains the partial derivative of the second function with respect to each variable, and so forth.  For this system, that becomes:

$$J = \begin{vmatrix}
3a^2 & e^{-b}\\
b & a - 2b
\end{vmatrix}$$

As a function:

    function jac(v)
        a = v[1]
        b = v[2]
        jac11 = 3*a^2
        jac12 = exp(-b)
        jac21 = b
        jac22 = a - 2*b
        jac   = [jac11 jac12; jac21 jac22]
    end

It is straightforward to provide a jacobian to your nlsolve algorithm; it is passed as a second function argument just after your objective function/system of equations

    solution = nlsolve(q, jac, [10.,10.])

Sometimes, it is of interest to benchmark the impact of changes you make.  There is a nice package for this called BenchmarkTools.jl

https://github.com/JuliaCI/BenchmarkTools.jl

Let's install it:

    using Pkg
    Pkg.add("BenchmarkTools")

To compare performance with and without our Jacobian, let's run:

    using BenchmarkTools
    @btime solution1 = nlsolve(q, [10.,10.])
    @btime solution2 = nlsolve(q, jac, [10., 10.])
    
You can see that it reduces time required considerably.  Although it is a non-issue for this easy problem, it can lead to a significant improvement in stability and performance for larger systems, so it is often worth the effort to generate a Jacobian for optimization and nonlinear system solutions.

# Problem Set

Solve the following problems using nlsolve

**Problem 1**

$$9x^2 + y^2 = 9$$
$$y^2 = 3x + 3$$

This is a nonlinear system of equations; you can easily solve this one analytically to verify your answer. Note that there is likely more than one solution; the (x,y) pair you find will depend on your initial guesses.  If you want to find all roots, solve it analytically or graphically.  For a graphical solution, solve each equation as y(x) and graph y vs. x.  You should be able to locate intersections, which correspond to solutions.

**Problem 2**

$$x^2 + 4y^2 = 9 - 3z$$
$$18y - 14x^2 + 45 = 0$$
$$x + y^2 - z^3 = 0$$

Again, you may have multiple solutions.  An analytical solution for x, y, and z is tractable here, as is a graphical solution (though it is a bit more of a brain bender).